# Урок 2. Метрики качества классификации ч.1

В уроке мы разбирали матрицу ошибок и некоторые метрики, основанные на терминах матрицы ошибок. Здесь же посмотрим, как эти метрики применять на практике и что  по ним можно выяснить.

### 1.1
Выкачайте тренировочный датасет Titanic - train.csv - с сайта [kaggle](https://www.kaggle.com/c/titanic/data). С помощью функции pd.read_csv() загрузите данные в датафрейм. Выведите первые 20 строк и проанализируйте данные: какие колонки присутствуют (более конкретная информация по ним есть на сайте kaggle), каким образом в них обозначены данные и какие типы данных используются (используйте pandas.dtypes). 

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


##### Словарь данных

| **Переменная** | **Определение**| **Значения** |
| --- | --- | --- |
| survival | Выживание | 0 = Нет, 1 = Да |
| pclass | Класс билета | 1 = 1-й, 2 = 2-й, 3 = 3-й |
| sex | Пол |  |
| Age | Возраст в годах |  |
| sibsp | Кол-во братьев и сестер / супругов на борту Титаника	 |  |
| parch | Кол-во родителей/детей на борту Титаника |  |
| ticket | Номер билета |  |
| fare | Пассажирский тариф |  |
| cabin | Номер каюты |  |
| embarked | Порт погрузки | C = Шербур, Q = Квинстаун, S = Саутгемптон |

In [4]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

### 1.2
Проверьте, имеются ли пропущенные значения в колонках, и выведите сумму всех пропущенных значений в каждой из колонок.

In [5]:
df.shape

(891, 12)

In [6]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### 1.3
Замените все пропущенные значения колонки Age на медианы в зависимости от пола человека: т.е. если пол человека в строке с пропущенным значением "male", заменяете пропущенное значение возраста на медиану по всем известным возрастам мужчин, и наоборот. Выведите медианы возраста в зависимости от пола. Пропущенные значения колонок Cabin и Embarked замените на U (Unknown).

Примечание: для группировки по полу можно использовать метод df.groupby(), который имеет встроенные агрегатные функции (в т.ч. для вычисления медианы).  Для установки пропущенных значений в датасете воспользуйтесь функцией df.apply().

In [7]:
age_median = dict()
age_median['male'] = df[df['Sex'] == 'male']['Age'].median()
age_median['female'] = df[df['Sex'] == 'female']['Age'].median()

print('Медиана возраста мужчин: {0}\n'
      'Медиана возраста женщин: {1}'.format(
          age_median['male'], age_median['female']
      ))

Медиана возраста мужчин: 29.0
Медиана возраста женщин: 27.0


In [8]:
## Лучше бойтись без apply

#df.loc[df['Sex'] == 'male', 'Age'] = df.loc[df['Sex'] == 'male', 'Age'].fillna(male_age_median)
#df.loc[df['Sex'] == 'female', 'Age'] = df.loc[df['Sex'] == 'female', 'Age'].fillna(female_age_median)

df.loc[(df['Sex'] == 'male') & df['Age'].isna(), 'Age'] = age_median['male']
df.loc[(df['Sex'] == 'female') & df['Age'].isna(), 'Age'] = age_median['female']

### 1.4
Выведите возраст пассажиров с PassengerID = [6, 20]. Убедитесь, что заполнены все пропущенные значения (воспользуйтесь функцией df.isnull() ).

In [9]:
df[(df['PassengerId'] == 6) | (df['PassengerId'] == 20)]['Age']

5     29.0
19    27.0
Name: Age, dtype: float64

In [10]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Так как не все классификаторы могут работать с категориальными признаками, в данном случае необходимо заменить их на числовые.

Также можно было бы выделить расширенные признаки, такие как статус пассажира (Mr., Mrs., Miss., Dr., Master. и т.д.), собственный индекс билета, индекс палубы и другие, но здесь ограничимся выбором более простых признаков.

### 1.5
В колонке Sex замените значения на 0, если пол "male", и на 1, если "female". В колонке Embarked замените параметры "U", "S", "C", "Q" на 0, 1, 2, 3 соответственно. Отбросьте колонки PassengerId, Name, Ticket, Cabin. Выведите первые 20 строк получившегося набора данных.

In [11]:
df.loc[(df['Sex'] == 'male'), 'Sex'] = 0
df.loc[(df['Sex'] == 'female'), 'Sex'] = 1
df['Sex'].unique()

array([0, 1], dtype=object)

In [12]:
df.loc[(df['Embarked'] == 'U'), 'Embarked'] = 0
df.loc[(df['Embarked'] == 'S'), 'Embarked'] = 1
df.loc[(df['Embarked'] == 'C'), 'Embarked'] = 2
df.loc[(df['Embarked'] == 'Q'), 'Embarked'] = 3
df['Embarked'].unique()

array([1, 2, 3, nan], dtype=object)

In [13]:
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
df.head(20)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,1
1,1,1,1,38.0,1,0,71.2833,2
2,1,3,1,26.0,0,0,7.9250,1
3,1,1,1,35.0,1,0,53.1000,1
4,0,3,0,35.0,0,0,8.0500,1
5,0,3,0,29.0,0,0,8.4583,3
6,0,1,0,54.0,0,0,51.8625,1
7,0,3,0,2.0,3,1,21.0750,1
8,1,3,1,27.0,0,2,11.1333,1
9,1,2,1,14.0,1,0,30.0708,2


In [14]:
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    2
dtype: int64

In [15]:
df.dropna(inplace=True)
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [16]:
df.to_csv('train_preprocessed.csv')

### 1.6
Приступим к построению классификаторов. **Условимся, что, если функция или объект модели имеют параметр random_state, то устанавливаем его равным 17 в каждом из случаев.**

Разделите данные на тренировочный и тестовый датасеты, установив размер тестового как 0.25 (первая колонка Survived является целевой, поэтому необходимо сначала ее отделить от признаков).

In [17]:
from sklearn.model_selection import train_test_split 

In [18]:
X = df.drop('Survived', axis=1)
y = df['Survived']

x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=17)

In [19]:
y.isna().sum()

0

### 1.7
Обучите на полученных выборках несколько классификаторов, которые импортированы ниже.

**Примечание: в методе ближайших соседей используйте количество соседей, равное 5.**

In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [21]:
schemes = dict(
    knn = KNeighborsClassifier(n_neighbors=5),
    gsnb = GaussianNB(),
    dtc = DecisionTreeClassifier(random_state=17),
    logreg = LogisticRegression(random_state=17),
)

models = dict()

for scheme_name, scheme in schemes.items():
  models[scheme_name] = scheme.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Очевидно, в этом случае использовать метрику accuracy, которую мы применяли до этого, не совсем правильно: она не даст верную оценку классификатору, потому как выживших явно меньше, чем погибших.

Вспомним про такие метрики как точность и полнота. Здесь они подойдут хорошо, так как у нас имеются положительный и отрицательный классы, и определить корректно один из них может оказаться более важным .

### 1.8
В каждом из пунктов следующего теста выберите, что важнее максимизировать: точность (precision) или полноту (recall). Выпишите ответы.

1. Вероятность того, что при определенной поломке самолета он сможет долететь до пункта назначения (1 - долетел, 0 - не долетел).
2. Предсказание, представляет ли человек опасность, по анализу психического состояния (1 - представляет опасность, 0 - не представляет опасности).
3. Предсказание ухода клиента (1 - клиент ушел, 0 - остался).
4. Выявление рака на основе медицинских показателей (1 - болен раком, 0 - здоров).
5. Предсказание летальности при наблюдаемой мутации (1 - выживание, 0 - летальный исход).
6. Определение важности происшествия для экстренных служб (1 - важно, 0 - неважно).
7. Окупятся ли вложения в бизнес (1 - окупятся, 0 - не окупятся).

Ваш ответ:

1. Precision<br>
2. Recall<br>
3. Recall<br>
4. Recall<br>
5. Precision<br>
6. Recall<br>
7. Precision<br>

### 1.9
Определите, что важнее использовать в случае датасета Titanic: полноту или точность. Объясните, почему.

*Ответ: в датасете Titanic мы предсказываем выживет ли человек. Поэтому нам важнее предсказать то, когда человек не выживет. Т.е надо избегать ложно положительных (FP) результатов (предсказаний того, что человек выживет, но да самом деле - нет) => необходимо использовать метрику Precision*

### 1.10
Для каждого классификатора выведите матрицу ошибок и самостоятельно рассчитайте метрику recall (расчеты должны присутствовать). Проверьте расчеты, воспользовавшись встроенной метрикой recall_score.

In [22]:
from sklearn.metrics import confusion_matrix, recall_score

In [23]:
predictions = dict()

for model_name, model in models.items():
  predictions[model_name] = model.predict(x_test)

In [24]:
def my_recall_score(y_true, y_pred, get_matrix=False):
  cm = confusion_matrix(y_true, y_pred)
  (tn, fp), (fn, tp) = cm

  if get_matrix:
    return tp / (tp + fn), cm
  else:
    return tp / (tp + fn)

In [25]:
recall_scores = dict()
confusion_matrixes = dict()

for prediction_model_name, prediction in predictions.items():
  my_rs, cm = my_recall_score(y_test, prediction, get_matrix=True)
  rs = recall_score(y_test, prediction)

  recall_scores[prediction_model_name] = rs
  confusion_matrixes[prediction_model_name] = cm

  print(f'Для {prediction_model_name}')
  print('rs == my_rs ?', rs == my_rs)
  print('rs =', rs)
  print('Матрица ошибок:\n', cm)
  print()

Для knn
rs == my_rs ? True
rs = 0.4774774774774775
Матрица ошибок:
 [[128  28]
 [ 58  53]]

Для gsnb
rs == my_rs ? True
rs = 0.7657657657657657
Матрица ошибок:
 [[132  24]
 [ 26  85]]

Для dtc
rs == my_rs ? True
rs = 0.7027027027027027
Матрица ошибок:
 [[129  27]
 [ 33  78]]

Для logreg
rs == my_rs ? True
rs = 0.7477477477477478
Матрица ошибок:
 [[136  20]
 [ 28  83]]



### 1.11
Выберите ту модель, на которой метрика recall давала лучший ответ, и рассчитайте для нее precision, используя встроенную функцию.

In [26]:
from sklearn.metrics import precision_score

In [27]:
best_model_by_recall_name = max(recall_scores, key=recall_scores.get)
best_model_by_recall_name, recall_scores[best_model_by_recall_name]

('gsnb', 0.7657657657657657)

In [28]:
best_model_by_recall_precision = precision_score(
    y_test, 
    predictions[best_model_by_recall_name]
)

best_model_by_recall_precision 

0.7798165137614679